# Task:
1. Simulate player gameplay 
2. Estimate their average spend
3. Determine fixed rewards for each completed level. The reward structure should be designed in such a way that the expected value of player rewards equals 1.3 times their
average spend (including any additional deposits).

### 1. Simulation of Gameplay
The player progresses through the levels by playing slot games. Upon reaching specific multipliers (e.g. x25, x50, etc.) in a limited time, the player advances to the next level. 

##### i. Player progression:
- Player enters a challenge at level 1 with a non zero bankroll (approx. 50 EUR)
- Players must hit or exceed the target multiplier in the given time.
- If they reached the next level the unused time is added to the next level.
- Players can add additional funds up to 4 times.

##### ii. End of the game:
- Players do not reach or surpass the target multiplaier in the given time
- Players reach the max multiplaier
- Players run out of money

### 2. Exploration of Variables

How results vary based on: (given a slot machine)
- initial bankroll size
- additional deposit (max 4)
- bet size: 1, 2, 5, 10, 20, 50 €

### 3. Reward Design

- Define a fixed monetary reward for each level.
- Ensure the prize structure logically scales with level difficulty and encourages
progression.
- The target is for the expected value of the rewards to equal 1.3× the player’s average
spend, including all deposits. Example: A player deposits a total €100 of their funds
into playing; the rewards should total €130.


### 4. Key metrics:

- player distribution across the levels
- avg. spend per level for all levels
- avg. time spend per level (potential indicator for player engagement)
- avg. reward per level



### 5. Assumptions:
- Spin time: 
    - 2 s (autospin)
    - 5 s (player input) 
- Bet size: 1, 2, 5, 10, 20, 50 € (just mimicking the currency options)

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import time


In [3]:
slot_machine_levels = {
    1: {
        "target_multiplier": 25,
        "time_limit": "30 min",
        "time_seconds": 30 * 60
    },
    2: {
        "target_multiplier": 50,
        "time_limit": "1 hour",
        "time_seconds": 1 * 60 * 60
    },
    3: {
        "target_multiplier": 100,
        "time_limit": "2 hours",
        "time_seconds": 2 * 60 * 60
    },
    4: {
        "target_multiplier": 200,
        "time_limit": "3 hours",
        "time_seconds": 3 * 60 * 60
    },
    5: {
        "target_multiplier": 500,
        "time_limit": "4 hours",
        "time_seconds": 4 * 60 * 60
    },
    6: {
        "target_multiplier": 800,
        "time_limit": "6 hours",
        "time_seconds": 6 * 60 * 60
    },
    7: {
        "target_multiplier": 1500,
        "time_limit": "8 hours",
        "time_seconds": 8 * 60 * 60
    },
    8: {
        "target_multiplier": 2500,
        "time_limit": "12 hours",
        "time_seconds": 12 * 60 * 60
    },
    9: {
        "target_multiplier": 5000,
        "time_limit": "24 hours",
        "time_seconds": 24 * 60 * 60
    }
}

print(slot_machine_levels)

slot1 = np.load('data/multipliers_slot_1.npy')
slot2 = np.load('data/multipliers_slot_2.npy')
print(type(slot1), slot1.shape)

{1: {'target_multiplier': 25, 'time_limit': '30 min', 'time_seconds': 1800}, 2: {'target_multiplier': 50, 'time_limit': '1 hour', 'time_seconds': 3600}, 3: {'target_multiplier': 100, 'time_limit': '2 hours', 'time_seconds': 7200}, 4: {'target_multiplier': 200, 'time_limit': '3 hours', 'time_seconds': 10800}, 5: {'target_multiplier': 500, 'time_limit': '4 hours', 'time_seconds': 14400}, 6: {'target_multiplier': 800, 'time_limit': '6 hours', 'time_seconds': 21600}, 7: {'target_multiplier': 1500, 'time_limit': '8 hours', 'time_seconds': 28800}, 8: {'target_multiplier': 2500, 'time_limit': '12 hours', 'time_seconds': 43200}, 9: {'target_multiplier': 5000, 'time_limit': '24 hours', 'time_seconds': 86400}}
<class 'numpy.ndarray'> (10000000,)


In [4]:
slot1_series = pd.Series(slot1, name="Slot 1")
slot2_series = pd.Series(slot2, name="Slot 2")

df_slot_stats = pd.concat([slot1_series, slot2_series], axis=1)

print(df_slot_stats.describe())


             Slot 1        Slot 2
count  1.000000e+07  2.000000e+07
mean   9.509598e-01  9.670817e-01
std    6.934455e+00  1.232296e+01
min    0.000000e+00  0.000000e+00
25%    0.000000e+00  0.000000e+00
50%    0.000000e+00  0.000000e+00
75%    1.666000e-01  0.000000e+00
max    7.000000e+02  1.470825e+04


In [5]:
remaining_time = 30*60 # s
t_spinn = 2 #s automatic spin, player click ~ 6 s
initial_budget = 50 #€
reached_lvl = 1
reached_multi = 0.0


def spin(slot):
    # slot = slot.flatten()
    random_value = np.random.choice(slot)
    print("pull", random_value)
    return random_value

def check_multi(level, multiplier, time_used):
    if level == slot_machine_levels[]:
        if multiplier==slot_machine_levels.target_multiplier:
            if time_used <= max_time_possible:
                return level+1  # the player won the lvl
    else:
        return 0 # the player lost the lvl and starts over again, reset all stats to the first level

i=8
while( i > 0):
    reached_multi += spin(slot1)
    print(reached_multi)
    print(i)
    # check multi
    i-=1




SyntaxError: invalid syntax (2852498460.py, line 15)

In [ ]:
def check_multi(level, current_multiplier, elapsed_seconds):
    """
    Returns:
      next_level (int) if current_multiplier >= target and elapsed_seconds <= time_limit
      1            if the player failed (reset to level 1)
    """
    info = slot_machine_levels.get(level)
    if not info:
        # invalid level: reset
        return 1

    if (current_multiplier >= info["target_multiplier"]
        and elapsed_seconds <= info["time_seconds"]):
        # success → go to next level (or stay at max)
        return min(level + 1, max(slot_machine_levels.keys()))
    else:
        # failure → reset
        return 1

def spin(slot):
    # slot = slot.flatten()
    random_value = np.random.choice(slot)
    print("pull", random_value)
    return random_value

def play_session(start_level=1):
    level = start_level
    iter_count = 0
    while level <= 2 and iter_count <100: #max(slot_machine_levels.keys()):
        iter_count +=1
        target = slot_machine_levels[level]["target_multiplier"]
        time_limit = slot_machine_levels[level]["time_seconds"]

        print(f"\n-- Level {level}: need ×{target} in {slot_machine_levels[level]['time_limit']}")

        current_multiplier = 0.0
        start_time = time.time()

        # spin until time runs out or target reached
        # while True:
        #     m = spin(slot1)
        #     current_multiplier += m
        #     elapsed = time.time() - start_time

        #     print(f"  Spin → ×{m:.2f} → total ×{current_multiplier:.2f}", end='\r')

        #     # check win/lose
        #     new_level = check_multi(level, current_multiplier, elapsed)
        #     if new_level > level:
        #         print(f"\n✅ Level {level} complete in {elapsed:.1f}s!")
        #         level = new_level
        #         break
        #     if elapsed > time_limit:
        #         print(f"\n❌ Time up! Resetting to level 1.")
        #         level = 1
        #         break

    print("\n🏆 You've conquered all levels!")

if __name__ == "__main__":
    play_session()

NameError: name 'slot_machine_levels' is not defined